# Part 1

## 1.1

In [1]:
#part1
import numpy as np      
import matplotlib.pyplot as plt 
import scipy.io.wavfile 
import subprocess
import librosa
import librosa.display
import IPython.display as ipd

from pathlib import Path, PurePath   
from tqdm.notebook import tqdm

import random
import pandas as pd
import sympy
import difflib
from scipy.spatial import distance
import os

#part3
from random import randint

### Utility functions

In [2]:
def convert_mp3_to_wav(audio:str) -> str:  
    """Convert an input MP3 audio track into a WAV file.

    Args:
        audio (str): An input audio track.

    Returns:
        [str]: WAV filename.
    """
    if audio[-3:] == "mp3":
        wav_audio = audio[:-3] + "wav"
        if not Path(wav_audio).exists():
                subprocess.check_output(f"ffmpeg -i {audio} {wav_audio}", shell=True)
        return wav_audio
    
    return audio

def plot_spectrogram_and_picks(track:np.ndarray, sr:int, peaks:np.ndarray, onset_env:np.ndarray) -> None:
    """[summary]

    Args:
        track (np.ndarray): A track.
        sr (int): Aampling rate.
        peaks (np.ndarray): Indices of peaks in the track.
        onset_env (np.ndarray): Vector containing the onset strength envelope.
    """
    times = librosa.frames_to_time(np.arange(len(onset_env)),
                            sr=sr, hop_length=HOP_SIZE)

    plt.figure()
    ax = plt.subplot(2, 1, 2)
    D = librosa.stft(track)
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(D), ref=np.max),
                            y_axis='log', x_axis='time')
    plt.subplot(2, 1, 1, sharex=ax)
    plt.plot(times, onset_env, alpha=0.8, label='Onset strength')
    plt.vlines(times[peaks], 0,
            onset_env.max(), color='r', alpha=0.8,
            label='Selected peaks')
    plt.legend(frameon=True, framealpha=0.8)
    plt.axis('tight')
    plt.tight_layout()
    plt.show()

def load_audio_picks(audio, duration, hop_size):
    """[summary]

    Args:
        audio (string, int, pathlib.Path or file-like object): [description]
        duration (int): [description]
        hop_size (int): 

    Returns:
        tuple: Returns the audio time series (track) and sampling rate (sr), a vector containing the onset strength envelope
        (onset_env), and the indices of peaks in track (peaks).
    """
    try:
        track, sr = librosa.load(audio, duration=duration)
        onset_env = librosa.onset.onset_strength(track, sr=sr, hop_length=hop_size)
        peaks = librosa.util.peak_pick(onset_env, 10, 10, 10, 10, 0.5, 0.5)
    except Error as e:
        print('An error occurred processing ', str(audio))
        print(e)

    return track, sr, onset_env, peaks

### Settings

In [3]:
N_TRACKS = 1413
HOP_SIZE = 512
DURATION = 60
THRESHOLD = 0 # TODO: to be tuned!

In [4]:
data_folder = Path("Part1/archive/mp3s-32k/")
mp3_tracks = data_folder.glob("*/*/*.mp3")
tracks = data_folder.glob("*/*/*.wav") 

### Preprocessing

In [5]:
for track in tqdm(mp3_tracks, total=N_TRACKS):
    convert_mp3_to_wav(str(track))

  0%|          | 0/1413 [00:00<?, ?it/s]

### Store peaks values and tracks titles

In [6]:
all_peaks=[]
titles=[]
t=open('titles.txt','w')
p=open('peaks60.txt','w')
for ind,audio in tqdm(enumerate(tracks)):
    track, sr, onset_env, peaks = load_audio_picks(audio, DURATION, HOP_SIZE)
    all_peaks.append(peaks)
    title=str(audio).replace('Part1/archive/mp3s-32k/','').replace('.wav','')
    #print(title)
    titles.append(title)
    t.write(title+'\n')
    p.write(str(peaks)+'\n')
t.close()
p.close()

0it [00:00, ?it/s]

## 1.2

### Create our Database

Create a dictionary with all the possible peaks as keys and values set to 0, we will use it to map the peaks previously stored into a space that is $\mathbb{R}^{k}$, with $k=\#\,total\, peaks$. Each of these vectors will be composed by only 0's and 1's, each value representing if the peak appears for that song (0=False, 1=True).

In [7]:
dic={}
for peaks in all_peaks:
    for peak in peaks:
        if peak not in dic:
            dic[peak]=0

The following function maps each of the peaks array (representing the peaks of a single song) into the space $\mathbb{R}^{k}$ and returns the 0-1 vector as a list.

In [8]:
def transform01(array,dic):
    
    dic=dict.fromkeys(dic, 0) #initialize to 0 all values, we wil call this func multiple tiems and don't want
                              #mistakes in storing the 
    
    for i in range(len(array)):
        dic[array[i]]=1
    

    return (list(dic.values()))

In [9]:
peaks_01=[]
for i in range(len(all_peaks)):
    peaks_01.append(transform01(all_peaks[i],dic))  #peaks_01 now contains all the 01 vectors

In [10]:
print(len(peaks_01),len(peaks_01[0])) #---> k=2579

1413 2579


As next step we want to reduce the dimensionality of these 0-1 vectors because they are very sparse, to do so we first define a scaling factor $scale$ (we are using $scale=3$) and then compress the vector by that factor as it follows:\
Imagine we have a vecotr $v=[1,0,1,0,0,0,1,1,1]$, we want it to become something like $v'=[1,0,1]$. To achieve this we defined a function that takes as input the scale factor and the vector, which is divided in subvectors each long $scale$. For each of those subvectors we return only 1 item that is the max of the elements in the subvector.

In [11]:
scale=2

In [12]:
def scaler(array,scale):
    scaled_01=[]
    for i in range(0,len(array),scale):
        summ=max(array[i:i+scale])
        scaled_01.append(summ)
    return np.asarray(scaled_01)

In [13]:
for i in range(len(peaks_01)):
    peaks_01[i]=scaler(peaks_01[i],scale)

In [14]:
print(len(peaks_01),len(peaks_01[0])) #---> k=1290

1413 1290


Now that a 0-1 vector for each song has been obtained, it will be used to produce a signature as follows:\
Repeat for $n_{permutation}=300$ times:

1.   Perform a random shuffling over the elements of the 0-1 vector;
1.   Store the index of the first "1" element into the signature for the song;

By doing this we are further reducing the dimensions of each vector from $\mathbb{R}^{k=1290}$ to $\mathbb{R}^{n_{permutation}=300}$, this will allow for faster operations on the vectors.

In [15]:
n_perms=300

In [16]:
def first1(array): #returns the index of the first '1' element of a np.array
    return np.where(array==1)[0][0]

In [17]:
def minHash(array,n_perms): #return list with indexes of first element of the given array
    
    indexes=[]
    random.seed(42) #set seed so we are applying the same transformation to the vectors when iterating(see later)
    
    for i in range(n_perms):
        random.shuffle(array)
        indexes.append(first1(array))
    return indexes

In [18]:
#now save in an array the values we get applying minHash on the whole list of vectors
new_indexes=[]
for i in tqdm(range(len(peaks_01))): #new_indexes will have as elements the columns of the signature matrix 
    indexes=minHash(peaks_01[i], n_perms) #we are trying to reduce the dimensions by a factor ~4, 
    new_indexes.append(np.asarray(indexes))    #that's why 300 perms 
    #new indexes is list of np.array

  0%|          | 0/1413 [00:00<?, ?it/s]

In [19]:
print(len(new_indexes[0]),len(new_indexes)) #we have 1413 vectors long 300 each, seems fine

300 1413


So now we have the list 'new_indexes' that contains the columns of the signature matrix, which is a matrix (300,1413) .

### Create Buckets for faster matching with queries

In this section we are dividing the signature matrix in bands, each band containing $bandwidth=4$ rows. Then we map each of the bands through a hash function into a value $v\,\epsilon\,(0,mod)$. The mod value used for the hash function is computed as follows:
* We first compute $m=k^{bandwidth}$, then we start increasing $m$ by 1 until when it is a prime number, when the condition is satisfied we assign $mod=m$. As such we choose $m$ to be the first prime number greater than $k *bandwidth$: we want to be able to represent at least $k*bandwidth$ combinations ($=\#(combinations\,each\,band\,can\, assume)$).

After this we store the data obtained in a dictionary as follows $data[v]=index$ where index refers to the index of the song, which we will use to retrieve the songs' titles later on when trying to match queries.

In [20]:
bandwidth=4

In [21]:
def findMod(l_01,bandwidth):
    m=l_01**bandwidth
    while(True):
        m+=1
        if(sympy.isprime(m)):
            break
    print("Module value chosen for the hash is: ",m)
    return(m)

In [22]:
mod=findMod(len(peaks_01[0]),bandwidth)

Module value chosen for the hash is:  2769228810013


In [23]:
#if I want to do with hash function.... choose prime number =1423
def rands(n,prime):
    a=[]
    random.seed(42)
    for i in range(n):
        a.append(random.randint(0, prime-1))
    return a

def hashf(A,a,prime,mod):
    a=np.asarray(a)
    A=np.asarray(A)
    return (np.sum(a*A))%prime

def mapHash(A,bandwidth,mod):
    prime=1423
    a=rands(bandwidth,prime)
    return hashf(A,a,prime,mod)

In [24]:
def Bucket1(array, data, bandwidth,mod):
    
    for j in tqdm(range(len(array))):
        
        for i in range(0,len(array[j]),bandwidth):
            
            key=mapHash(array[j][i:i+bandwidth],bandwidth,mod)
            
            if key not in data:
                data[key]=[]
                data[key].append(int(j))
            else:
                data[key].append(int(j))
    return data

In [25]:
data={}
data=dict.fromkeys(data, 0)
data=Bucket1(new_indexes,data,bandwidth,mod)

  0%|          | 0/1413 [00:00<?, ?it/s]

### Prepare to compare queries with dataset

In [26]:
def globsort(s):
    return int(os.path.basename(s)[5:-4])

In [27]:
query_folder = Path("Part1/queries/")
q_tracks = sorted(query_folder.glob("*.wav"),key=globsort)

Define a similarity score to compare the query with the elements in our dataset and a function to retrieve the title of a song given its index.

In [28]:
def sim_score(a1,a2): #returns value in interval (0,1) that states how similar two arrays are
    #return difflib.SequenceMatcher(None,a1,a2).ratio()
    return sum(a1==a2)/len(a1)
    
def retrieveTitle(index): #Return the title of a song given its index
    f=open('titles.txt')
    lines=f.readlines()
    f.close()
    return lines[index]

For each of the query songs we do the following:
1. Get the values of its peaks
1. Transform them into a 01 vector and scale by $scale$ factor
1. Apply minHash to retrieve a vector we called $sign\_colq$
1. Map to buckets
1. Compare the vector $sign\_colq$ with the possible vectors saved in the buckets (actually their index $index\,song$ is saved, we compare with $new_indexes$)
1. Pick the song with highest similarity value

### Analyze varying threshold

We introduce a threshold parameter for the queries, in case we want to look for similar songs to those of the query: when threshold < 1: songs in dataset that are similar to the query are reported for each track; when threshold $\equiv $1: songs in dataset that exactly match the query are reported for each track.

In [30]:
THRESHOLD=[0.3,0.8,1.]

for thresh in THRESHOLD:
    print('for threshold value={}'.format(thresh))
    for q_track in q_tracks:
        track, sr, onset_env, peakis = load_audio_picks(q_track, DURATION, HOP_SIZE)  
        q_01=np.array([0])
        q_01=peakis
        q_01=transform01(q_01,dic)                
        q_01=scaler(q_01,scale)
        sign_colq=minHash(q_01,n_perms)                                           
        
        keys=[]
        for i in range(0,len(sign_colq),bandwidth):                                   
            key=mapHash(sign_colq[i:i+bandwidth],bandwidth,mod)                              
            keys.append(key)  
            
        score=-1
        for key in keys:
            for value in data[key]:                                                   
                similarity=sim_score(sign_colq,new_indexes[value])
                if (similarity>score) & (similarity<=thresh):
                    best=value
                    score=similarity
        if score!=-1:                                                                  
            print(('Requested song of {} should be {}').format(str(q_track).replace('Part1/queries/','').replace('.wav','')
                                                                     ,retrieveTitle(best)))
        else:
            print(('threshold is too low, no match for {} in the buckets').format(str(q_track).replace('Part1/queries/','').replace('.wav','')))
    print('\n')

for threshold value=0.3
Requested song of track1 should be steely_dan/Countdown_To_Ecstasy/03-The_Boston_Rag

Requested song of track2 should be green_day/Warning/12-Macy_s_Day_Parade

Requested song of track3 should be prince/Parade/06-Life_Can_be_so_Nice

Requested song of track4 should be steely_dan/Katy_Lied/02-Bad_Sneakers

Requested song of track5 should be tori_amos/The_Beekeeper/12-Original_Sinsuality

Requested song of track6 should be prince/Parade/03-I_Wonder_U

Requested song of track7 should be radiohead/Pablo_Honey/04-Stop_Whispering

Requested song of track8 should be depeche_mode/Black_Celebration/02-Fly_on_the_windscreen

Requested song of track9 should be cure/The_Top/07-Piggy_In_The_Mirror

Requested song of track10 should be tori_amos/To_Venus_and_Back-Orbiting/09-Datura



for threshold value=0.8
Requested song of track1 should be steely_dan/Countdown_To_Ecstasy/03-The_Boston_Rag

Requested song of track2 should be green_day/Warning/12-Macy_s_Day_Parade

Requested 

# Part 2

# Part 3

You are given a list of integers, A, and another integer s. Write an algorithm that outputs all the pairs in A that equal s.
For example, if
A = [7, -2, 8, 2, 6, 4, -7, 2, 1, 3, -3] and s = 4
the algorithm should output: (7, -3), (-2, 6), (2, 2), (3, 1).

The easy way to solve the problem would be implementing a brute force method, which would lead to a complexity equal to $O(n^2)$.\
We managed to implement an algorithm that, making use of sort method of a list ($O(n\, log(n)$) and a cycle in which we use two indexes (more details in the code, below) solves the problem in $O(n\, log(n))$ [removing constants].

In [ ]:
def findPairs(A,s):
    pairs=[]
    A.sort                                   #First sort the array O(nlogn)
    start=0                                  #We define 2 indexes to go through the list A, and verify wheter the sum
    end=len(A)-1                             #of A[index1=start]+A[index2=end] is equal to the given sum, in the case it
                                             #is we store the value and keep looking for other values
                                             #if it's not we check if that's greater or not of the given sum:
    while start<end:                         #in the case it is we decrease the second index (end) while in the other
                                             #case we increase the first index (start)
        if A[start]+A[end]==s: 
            pairs.append((A[start],A[end]))
            start+=1
    
        elif A[start]+A[end]<s:
            start+=1
            
        else:
            end-=1
            
    return pairs

In [ ]:
N=10
for i in range(5):
    A=[]
    if i!=0: N*=10
    for i in range(N):
        A.append(randint(-50, 50))
    print(N)
    %timeit pairs=findPairs(A,10)